In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
from langchain_community.document_loaders import (
                                                  PyMuPDFLoader,
                                                  NewsURLLoader,
                                                  YoutubeLoader,
                                                  )
from langchain_text_splitters import RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
from src.utils import *
from src.ragqa import *

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = take_device()

Selected device: cuda


In [4]:
data = config_parser(data_config_path = 'config/data_config.yaml')

In [5]:
rag_system = RagSystem(data_config_path='config/model_config.yaml')

Selected device: cuda


/home/mrzaizai2k/code_Bao/LLM-with-RAG/venv/lib/python3.10/site-packages/faiss/loader.py:6: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [6]:
# rag_system.update_vector_db()
query="Tell me in detail The principles of software testing"
result = rag_system.final_result(query)
# print('result\n', result)
print(format_result(result=result))

result: 
1. Testing shows the presence of defects, not their absence
2. Exhaustive testing is impossible
3. Testing activities should start as early as possible
4. Defect clustering
5. The pesticide paradox
6. Testing is context dependent
7. No failures means the system is useful is a fallacy.
--page_content 0: Basic principles of software testing [2]
´ Principle 1: Testing shows the presence of defects, not their absence.

´ Principle 2: Exhaustive testing is impossible.

´ Principle 3: Testing activities should start as early as possible.

´ Principle 4: Defect clustering.

´ Principle 5: The pesticide paradox.

´ Principle 6: Testing is context dependent.

´ Principle 7: No failures means the system is useful is a fallacy.

Aug 2022
Software Testing - Introduction
8
--metadata 0: {'source': 'data/web_data/1_Introduction to Software Testing.pdf', 'file_path': 'data/web_data/1_Introduction to Software Testing.pdf', 'page': 7, 'total_pages': 15, 'format': 'PDF 1.4', 'title': '', 'autho

In [7]:
result

{'query': 'The principles of software testing',
 'source_documents': [Document(page_content='Basic principles of software testing [2]\n´ Principle 1: Testing shows the presence of defects, not their absence.\n\n´ Principle 2: Exhaustive testing is impossible.\n\n´ Principle 3: Testing activities should start as early as possible.\n\n´ Principle 4: Defect clustering.\n\n´ Principle 5: The pesticide paradox.\n\n´ Principle 6: Testing is context dependent.\n\n´ Principle 7: No failures means the system is useful is a fallacy.\n\nAug 2022\nSoftware Testing - Introduction\n8', metadata={'source': 'data/web_data/1_Introduction to Software Testing.pdf', 'file_path': 'data/web_data/1_Introduction to Software Testing.pdf', 'page': 7, 'total_pages': 15, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 13.4.1 (c) (Build 22F770820d) Quartz PDFContext', 'creationDate': "D:20240106045500Z00'00'", 'modDate': "D:20240106045500Z00'

In [8]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("BAAI/bge-m3")

In [11]:
def rerank_document_similarity(model, query, result, n_docs:int = None):
    be4_rank_docs  = [txt.page_content for txt in result['source_documents']]
    query_embedding = model.encode(query)
    passage_embedding = model.encode(be4_rank_docs)
    similarity_scores = util.dot_score(query_embedding, passage_embedding)
    sorted_scores, sorted_indices = torch.sort(similarity_scores, descending=True)
    print(sorted_scores, sorted_indices)
    # Sort the original list of texts based on the sorted indices
    result['source_documents'] = [result['source_documents'][idx] for idx in sorted_indices.tolist()[0]]
    if n_docs:
        result['source_documents'] = result['source_documents'][:n_docs]

    return result

In [14]:
result_2 = rerank_document_similarity(model, query, result)

tensor([[0.7841, 0.6325, 0.6091]]) tensor([[0, 1, 2]])


In [13]:
result_2

{'query': 'The principles of software testing',
 'source_documents': [Document(page_content='Basic principles of software testing [2]\n´ Principle 1: Testing shows the presence of defects, not their absence.\n\n´ Principle 2: Exhaustive testing is impossible.\n\n´ Principle 3: Testing activities should start as early as possible.\n\n´ Principle 4: Defect clustering.\n\n´ Principle 5: The pesticide paradox.\n\n´ Principle 6: Testing is context dependent.\n\n´ Principle 7: No failures means the system is useful is a fallacy.\n\nAug 2022\nSoftware Testing - Introduction\n8', metadata={'source': 'data/web_data/1_Introduction to Software Testing.pdf', 'file_path': 'data/web_data/1_Introduction to Software Testing.pdf', 'page': 7, 'total_pages': 15, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 13.4.1 (c) (Build 22F770820d) Quartz PDFContext', 'creationDate': "D:20240106045500Z00'00'", 'modDate': "D:20240106045500Z00'